### Importation of Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

### Loading the csv file

In [2]:
filename = r"‪‪‪‪C:\Users\Oluwayimika\Documents\Heart_data.csv"
df = pd.read_csv(filename)

OSError: [Errno 22] Invalid argument: '\u202a\u202a\u202a\u202aC:\\Users\\Oluwayimika\\Documents\\Heart_data.csv'

In [ ]:
df.head()

In [ ]:
df.rename(columns=
          {
              "cp": "chest_pain", "trestbps":"resting_bp", "restecg":"rest_ecg", "thalach":"max_hrt", "ca":"vessel"
          }, inplace=True
         )

#### Checking the shape of the dataset

In [ ]:
df.shape

#### Checking for Null values 

In [ ]:
df.isna().sum()

#### Filling up the null values

In [ ]:
for i in df.columns:
    if df[i].isna().sum() != 0:
        df[i] = df[i].fillna(method="ffill")

#### Checking for duplicate values

In [ ]:
df.duplicated().sum()

#### Dropping duplicate values

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

#### Checking for outliers

In [ ]:
count = 1
plt.figure(figsize=(9,5))
for i in df.drop(columns=["sex", "fbs", "exang", "target"]).columns.to_list():
    plt.subplot(4, 3, count)
    plt.boxplot(df[i])
    count += 1

#### Removing Outliers

In [ ]:
outliers_df = df.drop(columns=["sex", "fbs", "exang", "target"])
for i in outliers_df:
    min_quantile, max_quantile = df[i].quantile([0.1, 0.9])
    df = df[(df[i] >= min_quantile) & (df[i] <= max_quantile)]

In [ ]:
count = 1
plt.figure(figsize=(9,5))
for i in df.drop(columns=["sex", "fbs", "exang", "target"]).columns.to_list():
    print(i)
    plt.subplot(4, 3, count)
    plt.boxplot(df[i])
    count += 1

#### Fixing the target variable

In [ ]:
binary = []
non_binary = []
for i in df["target"]:
    if i < 2:
        binary.append(i)
    else:
        non_binary.append(i)

In [ ]:
len(binary) / (len(binary) + len(non_binary)) * 100

In [ ]:
def target(x):
    if x >= 1:
        return 1
    else:
        return 0
df["target"] = df["target"].map(target)

#### Checking if the data is balanced 

In [ ]:
df["target"].value_counts(normalize=True)

#### Balancing the imbalanced data using Imblearn library 

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
over_sampler =RandomOverSampler()

In [ ]:
X = df.drop(columns="target")
y = df["target"]

In [ ]:
X_over, y_over = over_sampler.fit_resample(X, y)

In [ ]:
sampled_df = pd.concat([X_over, y_over], axis=1)

In [ ]:
for i in sampled_df.drop(columns="oldpeak").columns.to_list():
    sampled_df[i] = sampled_df[i].astype(int)

### EDA

In [ ]:
count = 1
plt.figure(figsize=(14,14))
for i in sampled_df.drop(columns="target").columns.to_list():
    plt.subplot(4, 4, count)
    sampled_df[i].hist(bins = 50)
    plt.xlabel(i)
    count += 1

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(sampled_df.corr(), annot=True);

#### What gender has the highest number of persons with the heart disease

In [ ]:
sampled_df["sex"].groupby(sampled_df["target"][sampled_df["target"] == 1]).value_counts()

In [ ]:
sns.countplot(x=sampled_df["sex"], hue=sampled_df["target"])
plt.show()

#### What age category has the highest number of persons with the heart disease

In [ ]:
def age_category(x):
    if (x <=  71) & (x >= 65):
        return "old"
    elif (x <= 64) & (x >= 55):
        return "senior_adult"
    elif (x <= 54) & (x >= 45):
        return "adult"
    elif (x <= 44) & (x >= 35):
        return "young_adult"
sampled_df["age_category"] = sampled_df["age"].map(age_category)

In [ ]:
sampled_df["age_category"].groupby(sampled_df["target"]).value_counts()

In [ ]:
sns.countplot(x=sampled_df["age_category"], hue=sampled_df["target"])
plt.show()

#### Relationship between chest pain and the heart disease 

In [ ]:
sns.countplot(x=sampled_df["chest_pain"], hue=sampled_df["target"])
plt.show()

#### Relationship between resting blood pressure and the heart disease

In [ ]:
def bp_category(x):
    if (x <= 178) & (x >=  160):
        return "stage_2_hypertension"
    elif (x <= 159) & (x >= 140):
        return "stage_1_hypertension"
    elif (x <= 139) & (x >= 120):
        return "elevated"
    elif (x <= 119) & (x >= 100):
        return "normal"
sampled_df["resting_bp_cat"] = sampled_df["resting_bp"].map(bp_category)

In [ ]:
sns.countplot(x=sampled_df["resting_bp_cat"], hue=sampled_df["target"])
plt.show()

#### Relationship between resting bloodpressure and cholesterol

In [ ]:
sns.scatterplot(x=sampled_df["resting_bp"], y=sampled_df["chol"])
plt.show()

#### Relationship between chol and the heart disease

In [ ]:
def chol_category(x):
    if (x <= 360) & (x >=  281):
        return "very_high"
    elif (x <= 280) & (x >= 241):
        return "high"
    elif (x <= 240) & (x >= 201):
        return "moderate"
    elif (x <= 200) & (x >= 149):
        return "desirable"
sampled_df["chol_cat"] = sampled_df["chol"].map(chol_category)

In [ ]:
sns.countplot(x=sampled_df["chol_cat"], hue=sampled_df["target"])
plt.show()

#### Relationship between fasbt blood sugar and the heart disease

In [ ]:
sns.countplot(x=sampled_df["fbs"], hue=sampled_df["target"])

#### Relationship between rest_ecg and the heart disease

In [ ]:
sns.countplot(x=sampled_df["rest_ecg"], hue=sampled_df["target"])
plt.show()

### Data Preprocessing

In [ ]:
sampled_df["age_category"].replace(
    {"young_adult": 0, "adult": 1, "senior_adult": 2, "old": 3}, inplace=True
)

In [ ]:
sampled_df["resting_bp_cat"].replace(
    {"normal": 0, "elevated": 1, "stage_1_hypertension": 2, "stage_2_hypertension": 3}, inplace=True
)

In [ ]:
sampled_df["chol_cat"].replace(
    {"desirable": 0, "moderate": 1, "high": 2, "very_high": 3}, inplace=True
)

### Evaluation

#### Correlation coefficient of each variable

In [ ]:
corr_coef = {}
for i in sampled_df.drop(columns="target").columns.to_list():
    corr_coef[i] = sampled_df[i].corr(sampled_df["target"])
corr_coef

In [ ]:
for var, values in corr_coef.items():
    plt.barh(var, values)
    plt.title("Correlation Coefficient to the Target Variable")

In [ ]:
cols = ["age", "resting_bp_cat", "chol"]
sampled_df.drop(columns=cols, inplace=True)

In [ ]:
dataset = pd.get_dummies(
    sampled_df, 
    columns= ["sex", "fbs", "exang", "thal", "chest_pain", "rest_ecg", "slope", "vessel", "age_category", "chol_cat"]
)

In [ ]:
X = dataset.drop(columns="target")
y = dataset["target"]

In [ ]:
scaler = StandardScaler()
X[["max_hrt", "resting_bp", "oldpeak"]] = scaler.fit_transform(X[["max_hrt", "resting_bp", "oldpeak"]])

#### Splitting the data into training and testing sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Creating our model

In [ ]:
parameters = {
    "LogisticRegression": {
        "classifier": LogisticRegression(random_state=42),
        "params": {
            "penalty": ["l1", "l2"],
            "C": [0.001, 0.01, 0.1, 1, 10],
            "solver": ["liblinear", "saga"]
        }
    },
    "RandomForestClassifier": {
        "classifier": RandomForestClassifier(random_state=42),
        "params": {
            "n_estimators": [10, 20, 30, 40],
            "max_depth": [10, 20, 30],
            "min_samples_leaf": [1, 2, 3, 4],
            "min_samples_split": [2, 5, 7, 10]
        }
    },
    "GradientBoostingClassifier": {
        "classifier": GradientBoostingClassifier(random_state=42),
        "params": {
            "n_estimators": [50, 100, 150],
            "max_depth": [1, 3, 5,],
            "learning_rate": [0.01, 0.1, 0.5],
            "min_samples_split": [2, 5, 10]
        }
    },
    "DecisionTreeClassifier": {
        "classifier": DecisionTreeClassifier(random_state=42),
        "params": {
            "criterion": ["gini", "entropy"],
            "splitter": ["best", "random"],
            "max_depth": [10, 20, 30],
            "min_samples_leaf": [1, 2, 4],
            "min_samples_split": [2, 5, 10]
        }
    }
}

In [ ]:
performance = []
testing = []
for classifier, params in parameters.items():
    model = GridSearchCV(params["classifier"], params["params"], cv=5, verbose=10, scoring="roc_auc")
    
    model.fit(X_train, y_train)
    ypred = model.predict(X_test)
    
    performance.append({"Model": classifier, "Best Score": model.best_score_, "Best Parameters": model.best_params_})
    testing.append({"Model": classifier, "score": accuracy_score(ypred, y_test)})

[CV 4/5; 23/192] END max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=30;, score=0.865 total time=   0.0s
[CV 5/5; 23/192] START max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=30
[CV 5/5; 23/192] END max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=30;, score=0.898 total time=   0.0s
[CV 1/5; 24/192] START max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=40
[CV 1/5; 24/192] END max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=40;, score=0.844 total time=   0.1s
[CV 2/5; 24/192] START max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=40
[CV 2/5; 24/192] END max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=40;, score=0.803 total time=   0.1s
[CV 3/5; 24/192] START max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=40
[CV 3/5; 24/192] END max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=40;, score=0.800 total time=   0.1s

[CV 2/5; 31/192] END max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=30;, score=0.823 total time=   0.0s
[CV 3/5; 31/192] START max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=30
[CV 3/5; 31/192] END max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=30;, score=0.803 total time=   0.0s
[CV 4/5; 31/192] START max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=30
[CV 4/5; 31/192] END max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=30;, score=0.853 total time=   0.0s
[CV 5/5; 31/192] START max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=30
[CV 5/5; 31/192] END max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=30;, score=0.872 total time=   0.0s
[CV 1/5; 32/192] START max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=40
[CV 1/5; 32/192] END max_depth=10, min_samples_leaf=2, min_samples_split=10, n_estimators=40;, score=0.848 total tim

[CV 5/5; 38/192] END max_depth=10, min_samples_leaf=3, min_samples_split=5, n_estimators=20;, score=0.886 total time=   0.0s
[CV 1/5; 39/192] START max_depth=10, min_samples_leaf=3, min_samples_split=5, n_estimators=30
[CV 1/5; 39/192] END max_depth=10, min_samples_leaf=3, min_samples_split=5, n_estimators=30;, score=0.856 total time=   0.0s
[CV 2/5; 39/192] START max_depth=10, min_samples_leaf=3, min_samples_split=5, n_estimators=30
[CV 2/5; 39/192] END max_depth=10, min_samples_leaf=3, min_samples_split=5, n_estimators=30;, score=0.801 total time=   0.0s
[CV 3/5; 39/192] START max_depth=10, min_samples_leaf=3, min_samples_split=5, n_estimators=30
[CV 3/5; 39/192] END max_depth=10, min_samples_leaf=3, min_samples_split=5, n_estimators=30;, score=0.818 total time=   0.0s
[CV 4/5; 39/192] START max_depth=10, min_samples_leaf=3, min_samples_split=5, n_estimators=30
[CV 4/5; 39/192] END max_depth=10, min_samples_leaf=3, min_samples_split=5, n_estimators=30;, score=0.839 total time=   0.0s

[CV 5/5; 46/192] END max_depth=10, min_samples_leaf=3, min_samples_split=10, n_estimators=20;, score=0.883 total time=   0.0s
[CV 1/5; 47/192] START max_depth=10, min_samples_leaf=3, min_samples_split=10, n_estimators=30
[CV 1/5; 47/192] END max_depth=10, min_samples_leaf=3, min_samples_split=10, n_estimators=30;, score=0.837 total time=   0.0s
[CV 2/5; 47/192] START max_depth=10, min_samples_leaf=3, min_samples_split=10, n_estimators=30
[CV 2/5; 47/192] END max_depth=10, min_samples_leaf=3, min_samples_split=10, n_estimators=30;, score=0.807 total time=   0.0s
[CV 3/5; 47/192] START max_depth=10, min_samples_leaf=3, min_samples_split=10, n_estimators=30
[CV 3/5; 47/192] END max_depth=10, min_samples_leaf=3, min_samples_split=10, n_estimators=30;, score=0.805 total time=   0.0s
[CV 4/5; 47/192] START max_depth=10, min_samples_leaf=3, min_samples_split=10, n_estimators=30
[CV 4/5; 47/192] END max_depth=10, min_samples_leaf=3, min_samples_split=10, n_estimators=30;, score=0.836 total tim

[CV 5/5; 54/192] END max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=20;, score=0.873 total time=   0.0s
[CV 1/5; 55/192] START max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=30
[CV 1/5; 55/192] END max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=30;, score=0.842 total time=   0.0s
[CV 2/5; 55/192] START max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=30
[CV 2/5; 55/192] END max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=30;, score=0.812 total time=   0.0s
[CV 3/5; 55/192] START max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=30
[CV 3/5; 55/192] END max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=30;, score=0.817 total time=   0.0s
[CV 4/5; 55/192] START max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=30
[CV 4/5; 55/192] END max_depth=10, min_samples_leaf=4, min_samples_split=5, n_estimators=30;, score=0.843 total time=   0.0s

...

[CV 5/5; 71/108] END criterion=entropy, max_depth=10, min_samples_leaf=4, min_samples_split=10, splitter=best;, score=0.823 total time=   0.0s
[CV 1/5; 72/108] START criterion=entropy, max_depth=10, min_samples_leaf=4, min_samples_split=10, splitter=random
[CV 1/5; 72/108] END criterion=entropy, max_depth=10, min_samples_leaf=4, min_samples_split=10, splitter=random;, score=0.784 total time=   0.0s
[CV 2/5; 72/108] START criterion=entropy, max_depth=10, min_samples_leaf=4, min_samples_split=10, splitter=random
[CV 2/5; 72/108] END criterion=entropy, max_depth=10, min_samples_leaf=4, min_samples_split=10, splitter=random;, score=0.715 total time=   0.0s
[CV 3/5; 72/108] START criterion=entropy, max_depth=10, min_samples_leaf=4, min_samples_split=10, splitter=random
[CV 3/5; 72/108] END criterion=entropy, max_depth=10, min_samples_leaf=4, min_samples_split=10, splitter=random;, score=0.819 total time=   0.0s
[CV 4/5; 72/108] START criterion=entropy, max_depth=10, min_samples_leaf=4, min_

[CV 1/5; 80/108] END criterion=entropy, max_depth=20, min_samples_leaf=2, min_samples_split=2, splitter=random;, score=0.779 total time=   0.0s
[CV 2/5; 80/108] START criterion=entropy, max_depth=20, min_samples_leaf=2, min_samples_split=2, splitter=random
[CV 2/5; 80/108] END criterion=entropy, max_depth=20, min_samples_leaf=2, min_samples_split=2, splitter=random;, score=0.713 total time=   0.0s
[CV 3/5; 80/108] START criterion=entropy, max_depth=20, min_samples_leaf=2, min_samples_split=2, splitter=random
[CV 3/5; 80/108] END criterion=entropy, max_depth=20, min_samples_leaf=2, min_samples_split=2, splitter=random;, score=0.736 total time=   0.0s
[CV 4/5; 80/108] START criterion=entropy, max_depth=20, min_samples_leaf=2, min_samples_split=2, splitter=random
[CV 4/5; 80/108] END criterion=entropy, max_depth=20, min_samples_leaf=2, min_samples_split=2, splitter=random;, score=0.798 total time=   0.0s
[CV 5/5; 80/108] START criterion=entropy, max_depth=20, min_samples_leaf=2, min_sampl

[CV 1/5; 88/108] END criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_split=5, splitter=random;, score=0.753 total time=   0.0s
[CV 2/5; 88/108] START criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_split=5, splitter=random
[CV 2/5; 88/108] END criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_split=5, splitter=random;, score=0.710 total time=   0.0s
[CV 3/5; 88/108] START criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_split=5, splitter=random
[CV 3/5; 88/108] END criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_split=5, splitter=random;, score=0.785 total time=   0.0s
[CV 4/5; 88/108] START criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_split=5, splitter=random
[CV 4/5; 88/108] END criterion=entropy, max_depth=20, min_samples_leaf=4, min_samples_split=5, splitter=random;, score=0.746 total time=   0.0s
[CV 5/5; 88/108] START criterion=entropy, max_depth=20, min_samples_leaf=4, min_sampl

[CV 1/5; 96/108] END criterion=entropy, max_depth=30, min_samples_leaf=1, min_samples_split=10, splitter=random;, score=0.767 total time=   0.0s
[CV 2/5; 96/108] START criterion=entropy, max_depth=30, min_samples_leaf=1, min_samples_split=10, splitter=random
[CV 2/5; 96/108] END criterion=entropy, max_depth=30, min_samples_leaf=1, min_samples_split=10, splitter=random;, score=0.684 total time=   0.0s
[CV 3/5; 96/108] START criterion=entropy, max_depth=30, min_samples_leaf=1, min_samples_split=10, splitter=random
[CV 3/5; 96/108] END criterion=entropy, max_depth=30, min_samples_leaf=1, min_samples_split=10, splitter=random;, score=0.743 total time=   0.0s
[CV 4/5; 96/108] START criterion=entropy, max_depth=30, min_samples_leaf=1, min_samples_split=10, splitter=random
[CV 4/5; 96/108] END criterion=entropy, max_depth=30, min_samples_leaf=1, min_samples_split=10, splitter=random;, score=0.706 total time=   0.0s
[CV 5/5; 96/108] START criterion=entropy, max_depth=30, min_samples_leaf=1, mi

[CV 5/5; 103/108] END criterion=entropy, max_depth=30, min_samples_leaf=4, min_samples_split=2, splitter=best;, score=0.814 total time=   0.0s
[CV 1/5; 104/108] START criterion=entropy, max_depth=30, min_samples_leaf=4, min_samples_split=2, splitter=random
[CV 1/5; 104/108] END criterion=entropy, max_depth=30, min_samples_leaf=4, min_samples_split=2, splitter=random;, score=0.753 total time=   0.0s
[CV 2/5; 104/108] START criterion=entropy, max_depth=30, min_samples_leaf=4, min_samples_split=2, splitter=random
[CV 2/5; 104/108] END criterion=entropy, max_depth=30, min_samples_leaf=4, min_samples_split=2, splitter=random;, score=0.710 total time=   0.0s
[CV 3/5; 104/108] START criterion=entropy, max_depth=30, min_samples_leaf=4, min_samples_split=2, splitter=random
[CV 3/5; 104/108] END criterion=entropy, max_depth=30, min_samples_leaf=4, min_samples_split=2, splitter=random;, score=0.785 total time=   0.0s
[CV 4/5; 104/108] START criterion=entropy, max_depth=30, min_samples_leaf=4, min

In [ ]:
performance

In [ ]:
clf = GradientBoostingClassifier(n_estimators =50, learning_rate=0.1, max_depth=5, min_samples_split=2)
clf.fit(X_train, y_train)
ypred = clf.predict(X_test)
accuracy_score(ypred, y_test)

In [ ]:
clf = RandomForestClassifier(n_estimators=40, max_depth=10, min_samples_split=2, min_samples_leaf=1)
clf.fit(X_train, y_train)
ypred = clf.predict(X_test)
accuracy_score(ypred, y_test)